In [1]:
import pandas as pd

# Load metadata
metadata_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\archive (2)\data\token_metadata.csv"
metadata = pd.read_csv(metadata_path)

# Display some rows
print(metadata.head())

# Search for a specific coin (e.g., "Bitcoin")
print(metadata[metadata['name'].str.contains("Bitcoin", case=False)])


                                      address  decimals           name  \
0  0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2        18  Wrapped Ether   
1  0x2260fac5e5542a773aa44fbcfedf7c193bc2c599         8    Wrapped BTC   
2  0x1f9840a85d5af5bf1d1762f925bdaddc4201f984        18        Uniswap   
3  0x6b3595068778dd592e39a122f4f5a5cf09c90fe2        18     SushiToken   
4  0x8b3192f5eebd8579568a2ed41e6feb402f93f73f         9    Saitama Inu   

    symbol  
0     WETH  
1     WBTC  
2      UNI  
3    SUSHI  
4  SAITAMA  
                                         address  decimals             name  \
1046  0x309013d55fb0e8c17363bcc79f25d92f711a5802         9     Soft Bitcoin   
1339  0xb6ed7644c69416d67b522e20bc294a9a9b405b31         8  0xBitcoin Token   
1497  0xc34ef872a751a10e2a80243ef826ec0942fe3f14        18  Bitcoin Uniswap   
1829  0x8d717ab5eac1016b64c2a7fd04720fd2d27d1b86        18      BitcoinVend   

     symbol  
1046   SBTC  
1339  0xBTC  
1497   BTCu  
1829   BCVT  


In [22]:
def load_ohlc_by_symbol(symbol, metadata_df, base_path):
    row = metadata_df[metadata_df["symbol"].str.upper() == symbol.upper()]
    if row.empty:
        raise ValueError(f"Symbol {symbol} not found.")
    token_id = row.iloc[0].tolist()[0]
    file_path = fr"{base_path}\ohlc\{token_id}.csv"
    return pd.read_csv(file_path)

# Usage
base_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\archive (2)\data"
ohlc_df = load_ohlc_by_symbol("WBTC", metadata, base_path)
print(ohlc_df.head())


   price_open  price_high  price_low  price_close   timestamp  \
0     28162.3     28162.3    28162.2      28162.2  1680594960   
1     28161.1     28161.1    28161.1      28161.1  1680594780   
2     28141.9     28165.1    28141.9      28165.1  1680594720   
3     28148.3     28148.3    28148.3      28148.3  1680594660   
4     28130.2     28130.2    28126.4      28126.4  1680594540   

                  date  
0  2023-04-04T07:56:00  
1  2023-04-04T07:53:00  
2  2023-04-04T07:52:00  
3  2023-04-04T07:51:00  
4  2023-04-04T07:49:00  


In [24]:
df = ohlc_df.sort_values(by='date')

In [25]:
df.head()

,price_open,price_high,price_low,price_close,timestamp,date
749408,7711.14,7711.14,7711.14,7711.14,1589823900,2020-05-18T17:45:00
749407,7729.35,7729.35,7729.35,7729.35,1589824260,2020-05-18T17:51:00
749406,8316.49,8316.49,8316.49,8316.49,1589824980,2020-05-18T18:03:00
749405,9606.66,9606.66,9606.66,9606.66,1589825220,2020-05-18T18:07:00
749404,9531.21,9531.21,9531.21,9531.21,1589826120,2020-05-18T18:22:00


In [27]:
df.count()

price_open     749409
price_high     749409
price_low      749409
price_close    749409
timestamp      749409
date           749409
dtype: int64

In [28]:
# Ensure sorted by time
df = df.sort_values(by='timestamp')

# Calculate difference between consecutive timestamps
df['time_diff'] = df['timestamp'].diff()

# Get maximum irregularity
max_gap = df['time_diff'].max()
print(f"Maximum irregularity (in seconds): {max_gap}")

# Optional: convert to readable format
import datetime
print(f"That's approximately: {datetime.timedelta(seconds=max_gap)}")

Maximum irregularity (in seconds): 31380.0
That's approximately: 8:43:00


In [33]:
# Assuming you already have 'time_diff' column calculated as before
# Sort by time difference descending to get biggest gaps first
top_10_gaps = df.sort_values(by='time_diff', ascending=False).head(100)

# Show relevant columns (e.g., date and gap)
print(top_10_gaps)


        price_open  price_high  price_low  price_close   timestamp  \
748291     9673.44     9673.44    9673.44      9673.44  1591609260   
745434     9112.89     9112.89    9112.89      9112.89  1594382340   
749139     8972.07     8972.07    8972.07      8972.07  1590341640   
749284     9520.08     9520.08    9520.08      9520.08  1590039420   
749388     9508.21     9508.21    9508.21      9508.21  1589871660   
...            ...         ...        ...          ...         ...   
745438     9075.94     9075.94    9075.94      9075.94  1594353180   
748237     9648.88     9648.88    9648.88      9648.88  1591683720   
747837     9437.60     9437.60    9437.60      9437.60  1592438940   
743939     9656.69     9656.69    9656.69      9656.69  1595734380   
748188     9721.60     9721.60    9721.60      9721.60  1591789620   

                       date       time_diff  
748291  2020-06-08T09:41:00 0 days 08:43:00  
745434  2020-07-10T11:59:00 0 days 06:36:00  
749139  2020-05-24T17

In [32]:
import pandas as pd

# Suppose time_diff is in seconds (int or float)
df['time_diff'] = pd.to_timedelta(df['time_diff'], unit='s')

# Now you can compare with pd.Timedelta objects
gaps_over_1m = df[df['time_diff'] > pd.Timedelta(minutes=1)]
gaps_over_5m = df[df['time_diff'] > pd.Timedelta(minutes=5)]

print(f"Number of gaps > 1 minute: {len(gaps_over_1m)}")
print(f"Number of gaps > 5 minutes: {len(gaps_over_5m)}")


Number of gaps > 1 minute: 281851
Number of gaps > 5 minutes: 35293


In [35]:
df=pd.read_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\ETHUSD_1.csv")

In [40]:
df.head(10)

,1438956180,3.0,3.0.1,3.0.2,3.0.3,81.85727776,2
0,1438967940,3.00001,3.00001,3.00,3.00,42.073291,2
1,1439016180,3.00000,3.00000,3.00,3.00,0.400000,1
2,1439026260,2.00000,2.00000,2.00,2.00,125.000000,2
3,1439051400,1.20000,1.20000,1.20,1.20,54.759700,1
4,1439051460,1.20000,1.20000,1.20,1.20,53.747100,1
5,1439051580,1.20000,1.20000,1.20,1.20,39.627200,1
6,1439052060,1.20000,1.20000,1.20,1.20,51.866000,1
7,1439052900,1.02000,1.02000,1.02,1.02,34.445300,1
8,1439055240,1.02000,1.02000,1.00,1.00,247.252747,4
9,1439055300,1.00000,1.00000,1.00,1.00,29.659100,1


In [6]:
import pandas as pd

# Define the list of desired column names
column_names = ['Timestamp', 'Open', 'High', 'Low', 'Close','Volume', 'Trade_Count']

# Specify the path to your CSV file
file_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv"

# Read the CSV file into a pandas DataFrame
try:
    df = pd.read_csv(
        file_path,
        sep=',',
        skiprows=0,
        header=None,
        names=column_names
    )

    # Convert 'Timestamp' to datetime and store it in a new column called 'Datetime'
    df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='s')

    # Display the first few rows of the DataFrame and its info
    print("DataFrame Head:")
    print(df.head())

    print("\nDataFrame Info:")
    df.info()

except FileNotFoundError:
    print(f"Error: The file was not found at {file_path}")
except Exception as e:
    print(f"An error occurred: {e}")


DataFrame Head:
    Timestamp     Open     High  Low  Close      Volume  Trade_Count  \
0  1438956180  3.00000  3.00000  3.0    3.0   81.857278            2   
1  1438967940  3.00001  3.00001  3.0    3.0   42.073291            2   
2  1439016180  3.00000  3.00000  3.0    3.0    0.400000            1   
3  1439026260  2.00000  2.00000  2.0    2.0  125.000000            2   
4  1439051400  1.20000  1.20000  1.2    1.2   54.759700            1   

             Datetime  
0 2015-08-07 14:03:00  
1 2015-08-07 17:19:00  
2 2015-08-08 06:43:00  
3 2015-08-08 09:31:00  
4 2015-08-08 16:30:00  

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3743096 entries, 0 to 3743095
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   Timestamp    int64         
 1   Open         float64       
 2   High         float64       
 3   Low          float64       
 4   Close        float64       
 5   Volume       float64       
 6   Trade_Cou

In [7]:
df.sample(10)

,Timestamp,Open,High,Low,Close,Volume,Trade_Count,Datetime
816004,1542406380,170.50000,170.50000,170.35000,170.35000,137.639716,86,2018-11-16 22:13:00
458221,1514899080,854.41000,854.41000,853.00000,853.00000,24.358753,5,2018-01-02 13:18:00
2059495,1632848280,2828.00000,2828.00000,2819.94000,2820.14000,122.926937,27,2021-09-28 16:58:00
2441383,1657246440,1257.37000,1257.47000,1255.53000,1255.53000,4.359357,8,2022-07-08 02:14:00
1066960,1561110360,290.58000,290.77000,290.15000,290.28000,26.509000,13,2019-06-21 09:46:00
2143484,1638038760,4142.67000,4145.20000,4142.67000,4145.20000,1.591206,16,2021-11-27 18:46:00
1485130,1596297480,357.72000,358.00000,357.72000,358.00000,247.674409,30,2020-08-01 15:58:00
298101,1503770580,329.66954,329.66963,329.16251,329.16251,0.707294,4,2017-08-26 18:03:00
2875139,1687373400,1874.77000,1875.56000,1873.94000,1875.56000,18.911349,9,2023-06-21 18:50:00
2245882,1644478380,3183.26000,3187.24000,3183.26000,3187.24000,8.946515,14,2022-02-10 07:33:00


In [8]:
df.tail()

,Timestamp,Open,High,Low,Close,Volume,Trade_Count,Datetime
3743091,1743465240,1821.74,1821.74,1821.73,1821.74,40.534309,4,2025-03-31 23:54:00
3743092,1743465300,1821.74,1822.83,1821.74,1822.83,2.596766,4,2025-03-31 23:55:00
3743093,1743465420,1822.98,1823.49,1822.98,1823.49,0.091093,3,2025-03-31 23:57:00
3743094,1743465480,1823.06,1823.61,1823.06,1823.41,4.131870,6,2025-03-31 23:58:00
3743095,1743465540,1822.88,1822.88,1822.80,1822.80,7.307224,6,2025-03-31 23:59:00


In [10]:
import pandas as pd
# Ensure sorted by time

df = df.sort_values(by='Timestamp')

# Calculate difference between consecutive timestamps
df['time_diff'] = df['Timestamp'].diff()

# Get maximum irregularity
max_gap = df['time_diff'].max()
print(f"Maximum irregularity (in seconds): {max_gap}")

# Optional: convert to readable format
import datetime
print(f"That's approximately: {datetime.timedelta(seconds=max_gap)}")
# Suppose time_diff is in seconds (int or float)
df['time_diff'] = pd.to_timedelta(df['time_diff'], unit='s')

# Now you can compare with pd.Timedelta objects
gaps_over_1m = df[df['time_diff'] > pd.Timedelta(minutes=10)]
gaps_over_5m = df[df['time_diff'] > pd.Timedelta(minutes=5)]

print(f"Number of gaps > 1 minute: {len(gaps_over_1m)}")
print(f"Number of gaps > 5 minutes: {len(gaps_over_5m)}")


Maximum irregularity (in seconds): 185700.0
That's approximately: 2 days, 3:35:00
Number of gaps > 1 minute: 17381
Number of gaps > 5 minutes: 41012


In [ ]:
import pandas as pd

# Load your CSV file
file_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\raw\XBTEUR_1.csv"

# Define column names
column_names = ['Index', 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Trade_Count']

# Read the CSV
df = pd.read_csv(file_path, names=column_names, header=None, skiprows=1)

# Convert Unix timestamp to datetime in a new column
df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='s')

# Set 'Datetime' as the index for resampling
df.set_index('Datetime', inplace=True)

# uncomment only when sampling for 15 min or 1 hour is needed 
# # Resample to hourly frequency and aggregate (e.g., OHLC, sum for volume/trades)
# df_hourly = df.resample('1M').agg({
#     'Open': 'first',
#     'High': 'max',
#     'Low': 'min',
#     'Close': 'last',
#     'Volume': 'sum',
#     'Trade_Count': 'sum'
# }).dropna()  # drop rows with incomplete data

# # Reset index if needed
# df_hourly = df_hourly.reset_index()

# # Show result
# print(df_hourly.head())


C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_3636\3352476562.py:19: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df_hourly = df.resample('1M').agg({


    Datetime  Open      High   Low      Close    Volume  Trade_Count
0 1970-01-31  81.0  105487.5  81.0  13.801311  94565395          0.0


Hourly data saved to: C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETHhourly.csv


In [2]:
import pandas as pd
# Ensure sorted by time
df = df.sort_values(by='Timestamp')

# Calculate difference between consecutive timestamps
df['time_diff'] = df['Timestamp'].diff()

# Get maximum irregularity
max_gap = df['time_diff'].max()
print(f"Maximum irregularity (in seconds): {max_gap}")

# Optional: convert to readable format
import datetime
print(f"That's approximately: {datetime.timedelta(seconds=max_gap)}")
# Suppose time_diff is in seconds (int or float)
df['time_diff'] = pd.to_timedelta(df['time_diff'], unit='s')

# Now you can compare with pd.Timedelta objects
gaps_over_1m = df[df['time_diff'] > pd.Timedelta(minutes=60)]
gaps_over_5m = df[df['time_diff'] > pd.Timedelta(minutes=5)]

print(f"Number of gaps > 1 minute: {len(gaps_over_1m)}")
print(f"Number of gaps > 5 minutes: {len(gaps_over_5m)}")


Maximum irregularity (in seconds): 4.600000000000364
That's approximately: 0:00:04.600000
Number of gaps > 1 minute: 0
Number of gaps > 5 minutes: 0


In [ ]:
# import pandas as pd
# import datetime

# def check_hourly_gaps(file_path, timestamp_col='Timestamp'):
#     try:
#         # Load CSV file
#         df = pd.read_csv(file_path, names=column_names, header=None, skiprows=1)

#         # Ensure the timestamp column is present
#         if timestamp_col not in df.columns:
#             raise ValueError(f"Column '{timestamp_col}' not found in the dataset.")

#         # Sort by timestamp
#         df = df.sort_values(by=timestamp_col)

#         # Calculate time difference
#         df['time_diff'] = df[timestamp_col].diff()

#         # If timestamp is Unix (numeric), convert to timedelta
#         if pd.api.types.is_numeric_dtype(df['time_diff']):
#             df['time_diff'] = pd.to_timedelta(df['time_diff'], unit='s')

#         # Find maximum irregularity
#         max_gap = df['time_diff'].max()
#         print(f"Maximum irregularity: {max_gap} ({max_gap.total_seconds()} seconds)")

#         # Filter rows with gaps over 1 hour
#         gaps_over_1h = df[df['time_diff'] > pd.Timedelta(hours=1)]
#         print(f"Number of gaps > 1 hour: {len(gaps_over_1h)}")

#         # Optional: show top 5 gaps
#         if not gaps_over_1h.empty:
#             print("\nTop gaps over 1 hour:")
#             print(gaps_over_1h[[timestamp_col, 'time_diff']].head(5))
#         else:
#             print("No gaps over 1 hour found.")

#     except FileNotFoundError:
#         print(f"File not found: {file_path}")
#     except Exception as e:
#         print(f"Error: {e}")

# # === USAGE EXAMPLE ===
# file_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv"
# check_hourly_gaps(file_path)


Maximum irregularity: 2 days 03:35:00 (185700.0 seconds)
Number of gaps > 1 hour: 1373

Top gaps over 1 hour:
                      Timestamp       time_diff
2015-08-08 06:43:00  1439016180 0 days 13:24:00
2015-08-08 09:31:00  1439026260 0 days 02:48:00
2015-08-08 16:30:00  1439051400 0 days 06:59:00
2015-08-08 21:41:00  1439070060 0 days 03:58:00
2015-08-11 01:16:00  1439255760 2 days 03:35:00


In [13]:
import pandas as pd
# Load your CSV file
file_path = r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\raw\XBTEUR_1.csv"

# Define column names
column_names = [ 'Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume', 'Trade_Count']

# Read the CSV
df = pd.read_csv(file_path, names=column_names, header=None)





df

,Timestamp,Open,High,Low,Close,Volume,Trade_Count
0,1378856820,97.00,97.00,97.0,97.0,1.000000,1
1,1378859580,99.90,99.90,99.9,99.9,0.100000,1
2,1378859640,99.90,99.90,99.9,99.9,0.100000,1
3,1378869720,98.20,98.20,98.2,98.2,0.100000,1
4,1378875000,96.91,96.91,96.9,96.9,1.000000,2
...,...,...,...,...,...,...,...
5172687,1743465300,76291.50,76291.50,76284.2,76284.2,0.010191,7
5172688,1743465360,76291.40,76291.50,76291.4,76291.5,0.002511,3
5172689,1743465420,76310.60,76314.30,76306.6,76306.6,0.006406,6
5172690,1743465480,76320.80,76322.70,76320.8,76321.4,0.011243,4


In [14]:

df['ReadableTime'] = pd.to_datetime(df['Timestamp'], unit='s')
df = df.set_index('ReadableTime')
df = df.sort_index()

# Check for duplicates
num_dupes = df.index.duplicated().sum()
print(f"Number of duplicate timestamps: {num_dupes}")




Number of duplicate timestamps: 0


In [15]:
df.head()

,Timestamp,Open,High,Low,Close,Volume,Trade_Count
ReadableTime,,,,,,,
2013-09-10 23:47:00,1378856820,97.00,97.00,97.0,97.0,1.0,1
2013-09-11 00:33:00,1378859580,99.90,99.90,99.9,99.9,0.1,1
2013-09-11 00:34:00,1378859640,99.90,99.90,99.9,99.9,0.1,1
2013-09-11 03:22:00,1378869720,98.20,98.20,98.2,98.2,0.1,1
2013-09-11 04:50:00,1378875000,96.91,96.91,96.9,96.9,1.0,2


In [16]:

# # Convert Unix timestamp to datetime in a new column
# df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='s')

# # Set 'Datetime' as the index for resampling
# df.set_index('Datetime', inplace=True)

# # Resample to hourly frequency and aggregate (e.g., OHLC, sum for volume/trades)
# # df_hourly = df.resample('1H').agg({
# #     'Open': 'first',
# #     'High': 'max',
# #     'Low': 'min',
# #     'Close': 'last',
# #     'Volume': 'sum',
# #     'Trade_Count': 'sum'
# # }).dropna()  # drop rows with incomplete data

# # Reset index if needed
# # df_hourly = df_hourly.reset_index()

# # # Show result
# # print(df_hourly.head())
# # df_hourly.to_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv")

In [17]:
df.head()

,Timestamp,Open,High,Low,Close,Volume,Trade_Count
ReadableTime,,,,,,,
2013-09-10 23:47:00,1378856820,97.00,97.00,97.0,97.0,1.0,1
2013-09-11 00:33:00,1378859580,99.90,99.90,99.9,99.9,0.1,1
2013-09-11 00:34:00,1378859640,99.90,99.90,99.9,99.9,0.1,1
2013-09-11 03:22:00,1378869720,98.20,98.20,98.2,98.2,0.1,1
2013-09-11 04:50:00,1378875000,96.91,96.91,96.9,96.9,1.0,2


In [19]:
# Strip whitespace from column headers
df.columns = df.columns.str.strip()

# Now try again
df['Datetime'] = pd.to_datetime(df['Timestamp'], unit='s')
df.set_index('Datetime', inplace=True)
df.sort_index(inplace=True)

# Resample to 1-hour frequency and forward fill
# df_resampled = df.resample('1H').asfreq()
df = df.ffill()




In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5172692 entries, 2013-09-10 23:47:00 to 2025-03-31 23:59:00
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   Timestamp    int64  
 1   Open         float64
 2   High         float64
 3   Low          float64
 4   Close        float64
 5   Volume       float64
 6   Trade_Count  int64  
dtypes: float64(5), int64(2)
memory usage: 315.7 MB


# new DataSet


In [19]:
import pandas as pd


# Load the timestamps
df = pd.read_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\raw\XBTEUR_1.csv", names=["timestamp", 'open', 'high', 'low', 'close', 'volume', 'trades'])

df

,timestamp,open,high,low,close,volume,trades
0,1378856820,97.00,97.00,97.0,97.0,1.000000,1
1,1378859580,99.90,99.90,99.9,99.9,0.100000,1
2,1378859640,99.90,99.90,99.9,99.9,0.100000,1
3,1378869720,98.20,98.20,98.2,98.2,0.100000,1
4,1378875000,96.91,96.91,96.9,96.9,1.000000,2
...,...,...,...,...,...,...,...
5172687,1743465300,76291.50,76291.50,76284.2,76284.2,0.010191,7
5172688,1743465360,76291.40,76291.50,76291.4,76291.5,0.002511,3
5172689,1743465420,76310.60,76314.30,76306.6,76306.6,0.006406,6
5172690,1743465480,76320.80,76322.70,76320.8,76321.4,0.011243,4


In [20]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df = df.sort_values('timestamp')
df.set_index('timestamp', inplace=True)

In [21]:
df

,open,high,low,close,volume,trades
timestamp,,,,,,
2013-09-10 23:47:00,97.00,97.00,97.0,97.0,1.000000,1
2013-09-11 00:33:00,99.90,99.90,99.9,99.9,0.100000,1
2013-09-11 00:34:00,99.90,99.90,99.9,99.9,0.100000,1
2013-09-11 03:22:00,98.20,98.20,98.2,98.2,0.100000,1
2013-09-11 04:50:00,96.91,96.91,96.9,96.9,1.000000,2
...,...,...,...,...,...,...
2025-03-31 23:55:00,76291.50,76291.50,76284.2,76284.2,0.010191,7
2025-03-31 23:56:00,76291.40,76291.50,76291.4,76291.5,0.002511,3
2025-03-31 23:57:00,76310.60,76314.30,76306.6,76306.6,0.006406,6


In [22]:
ohlc_dict = {
    'open': 'first',
    'high': 'max',
    'low': 'min',
    'close': 'last',
    'volume': 'sum',
    'trades': 'sum'
}

# df_hourly = df.resample('H').apply(ohlc_dict)


In [23]:
df_hourly =df.interpolate(method='time')


In [24]:
import numpy as np

epsilon = 1e-10  # Small constant to prevent divide by zero

for col in ['open', 'high', 'low', 'close', 'volume', 'trades']:
    # Replace 0 with epsilon to avoid log(0)
    safe_current = df_hourly[col].replace(0, epsilon)
    safe_prev = df_hourly[col].shift(1).replace(0, epsilon)
    
    df_hourly[f'{col}_log_returns'] = np.log(safe_current / safe_prev)


In [25]:
df_hourly

,open,high,low,close,volume,trades,open_log_returns,high_log_returns,low_log_returns,close_log_returns,volume_log_returns,trades_log_returns
timestamp,,,,,,,,,,,,
2013-09-10 23:47:00,97.00,97.00,97.0,97.0,1.000000,1,NaN,NaN,NaN,NaN,NaN,NaN
2013-09-11 00:33:00,99.90,99.90,99.9,99.9,0.100000,1,0.029459,0.029459,0.029459,0.029459,-2.302585,0.000000
2013-09-11 00:34:00,99.90,99.90,99.9,99.9,0.100000,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2013-09-11 03:22:00,98.20,98.20,98.2,98.2,0.100000,1,-0.017163,-0.017163,-0.017163,-0.017163,0.000000,0.000000
2013-09-11 04:50:00,96.91,96.91,96.9,96.9,1.000000,2,-0.013224,-0.013224,-0.013327,-0.013327,2.302585,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-31 23:55:00,76291.50,76291.50,76284.2,76284.2,0.010191,7,0.000406,0.000000,0.000311,-0.000096,1.007981,0.336472
2025-03-31 23:56:00,76291.40,76291.50,76291.4,76291.5,0.002511,3,-0.000001,0.000000,0.000094,0.000096,-1.400806,-0.847298
2025-03-31 23:57:00,76310.60,76314.30,76306.6,76306.6,0.006406,6,0.000252,0.000299,0.000199,0.000198,0.936621,0.693147


In [26]:
df_hourly['hour'] = df_hourly.index.hour


In [27]:
import pandas as pd
import numpy as np

# --- DEMA ---
def dema(series, span):
    ema1 = series.ewm(span=span, adjust=False).mean()
    ema2 = ema1.ewm(span=span, adjust=False).mean()
    return 2 * ema1 - ema2

df_hourly['dema'] = dema(df_hourly['close'], span=20)

# --- VWAP ---
df_hourly['vwap'] = (df_hourly['close'] * df_hourly['volume']).cumsum() / df_hourly['volume'].cumsum()

# --- Bollinger Bands ---
window = 20
rolling_mean = df_hourly['close'].rolling(window)
df_hourly['bband_up'] = rolling_mean.mean() + 2 * rolling_mean.std()
df_hourly['bband_down'] = rolling_mean.mean() - 2 * rolling_mean.std()

# --- ADL ---
mfm = ((df_hourly['close'] - df_hourly['low']) - (df_hourly['high'] - df_hourly['close'])) / (df_hourly['high'] - df_hourly['low']).replace(0, np.nan)
mfv = mfm * df_hourly['volume']
df_hourly['adl'] = mfv.cumsum()

# --- OBV ---
df_hourly['obv'] = (np.sign(df_hourly['close'].diff()) * df_hourly['volume']).fillna(0).cumsum()

# --- MACD ---
ema12 = df_hourly['close'].ewm(span=12, adjust=False).mean()
ema26 = df_hourly['close'].ewm(span=26, adjust=False).mean()
df_hourly['macd'] = ema12 - ema26
df_hourly['macd_signal'] = df_hourly['macd'].ewm(span=9, adjust=False).mean()
df_hourly['macd_diff'] = df_hourly['macd'] - df_hourly['macd_signal']

# --- Stochastic Oscillator ---
low14 = df_hourly['low'].rolling(14).min()
high14 = df_hourly['high'].rolling(14).max()
df_hourly['stoch_k'] = 100 * ((df_hourly['close'] - low14) / (high14 - low14))
df_hourly['stoch_d'] = df_hourly['stoch_k'].rolling(3).mean()

# --- Aroon ---
window = 25
df_hourly['aroon_up'] = df_hourly['high'].rolling(window).apply(lambda x: 100 * (window - np.argmax(x[::-1])) / window, raw=True)
df_hourly['aroon_down'] = df_hourly['low'].rolling(window).apply(lambda x: 100 * (window - np.argmin(x[::-1])) / window, raw=True)

# --- RSI ---
delta = df_hourly['close'].diff()
gain = delta.where(delta > 0, 0)
loss = -delta.where(delta < 0, 0)
avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()
rs = avg_gain / avg_loss
df_hourly['rsi'] = 100 - (100 / (1 + rs))

#--- ADX ---
def calculate_adx(df, window=14):
    high = df['high']
    low = df['low']
    close = df['close']

    plus_dm = high.diff()
    minus_dm = low.diff()

    plus_dm = pd.Series(np.where((plus_dm > minus_dm) & (plus_dm > 0), plus_dm, 0.0), index=df.index)
    minus_dm = pd.Series(np.where((minus_dm > plus_dm) & (minus_dm > 0), minus_dm, 0.0), index=df.index)

    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)

    atr = tr.rolling(window).mean()

    plus_di = 100 * (plus_dm.rolling(window).mean() / atr)
    minus_di = 100 * (minus_dm.rolling(window).mean() / atr)

    dx = 100 * abs(plus_di - minus_di) / ((plus_di + minus_di).replace(0, np.nan))
    adx = dx.rolling(window).mean()

    return adx


df_hourly['adx'] = calculate_adx(df_hourly)

# --- CCI ---
tp = (df_hourly['high'] + df_hourly['low'] + df_hourly['close']) / 3
sma = tp.rolling(20).mean()
mad = tp.rolling(20).apply(lambda x: np.mean(np.abs(x - np.mean(x))), raw=True)
df_hourly['cci'] = (tp - sma) / (0.015 * mad)


In [28]:
df_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5172692 entries, 2013-09-10 23:47:00 to 2025-03-31 23:59:00
Data columns (total 29 columns):
 #   Column              Dtype  
---  ------              -----  
 0   open                float64
 1   high                float64
 2   low                 float64
 3   close               float64
 4   volume              float64
 5   trades              int64  
 6   open_log_returns    float64
 7   high_log_returns    float64
 8   low_log_returns     float64
 9   close_log_returns   float64
 10  volume_log_returns  float64
 11  trades_log_returns  float64
 12  hour                int64  
 13  dema                float64
 14  vwap                float64
 15  bband_up            float64
 16  bband_down          float64
 17  adl                 float64
 18  obv                 float64
 19  macd                float64
 20  macd_signal         float64
 21  macd_diff           float64
 22  stoch_k             float64
 23  stoch_d             floa

In [13]:
from pytrends.request import TrendReq
import pandas as pd

pytrends = TrendReq(hl='en-US', tz=0)
pytrends.build_payload([ 'Bitcoin Price'],  timeframe='2013-08-07 2025-03-31')


df_trends = pytrends.interest_over_time().drop(columns=['isPartial'])
df_trends.index = pd.to_datetime(df_trends.index, utc=True)
df_trends.rename(columns={'ETH USD': 'trends'}, inplace=True)


In [16]:
import pandas as pd

# Assuming your daily data looks like this:
# df_trends (index is daily datetime, column 'trends')

# First, make sure the index is a datetime index at daily frequency
df_trends.index = pd.to_datetime(df_trends.index)
df_trends = df_trends.asfreq('D')  # enforce daily frequency

# Upsample to hourly frequency, then interpolate values linearly
df_hourly_trends = df_trends.resample('T').interpolate(method='linear')




In [30]:
df_hourly_trends.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6091201 entries, 2013-08-01 00:00:00+00:00 to 2025-03-01 00:00:00+00:00
Freq: T
Data columns (total 1 columns):
 #   Column         Dtype  
---  ------         -----  
 0   Bitcoin Price  float64
dtypes: float64(1)
memory usage: 92.9 MB


In [37]:
# Ensure both indices are datetime and in UTC
df_hourly.index = pd.to_datetime(df_hourly.index, utc=True)
df_hourly_trends.index = pd.to_datetime(df_hourly_trends.index, utc=True)

# Reset index from 'datetime' for merging
df_hourly_reset = df_hourly.reset_index().rename(columns={'datetime': 'timestamp'})
df_trends_reset = df_hourly_trends.reset_index().rename(columns={'date': 'timestamp'})  # if df_trends index is 'date'

# Merge using merge_asof on timestamp
df_merged = pd.merge_asof(
    df_hourly_reset.sort_values('timestamp'),
    df_trends_reset.sort_values('timestamp'),
    on='timestamp',
    direction='nearest',
    tolerance=pd.Timedelta('1T')
)

# Set timestamp back as index and rename to original if needed
df_hourly = df_merged.set_index('timestamp')
df_hourly.index.name = 'datetime'  # if you want the index name restored


In [33]:
# # Make sure both dataframes have datetime indices (and compatible timezone info)
# df_hourly.index = pd.to_datetime(df_hourly.index)
# df_hourly_trends.index = pd.to_datetime(df_hourly_trends.index)

# # If one is timezone-aware and the other isn't, you might need to align:
# # Example: make both timezone-aware UTC:
# df_hourly.index = df_hourly.index.tz_convert('UTC') if df_hourly.index.tzinfo else df_hourly.index.tz_localize('UTC')
# df_hourly_trends.index = df_hourly_trends.index.tz_convert('UTC') if df_hourly_trends.index.tzinfo else df_hourly_trends.index.tz_localize('UTC')

# # Merge the trends column into df_hourly by index
# df_hourly = df_hourly.merge(df_hourly_trends[['trends']], left_index=True, right_index=True, how='left')

# # Now df_hourly has a new 'trends' column with matching hourly trends where available.


In [39]:
df_hourly.drop(columns=['trends'], inplace=True, errors='ignore')

In [40]:
df_hourly.sample(10)

,open,high,low,close,volume,trades,open_log_returns,high_log_returns,low_log_returns,close_log_returns,...,macd_diff,stoch_k,stoch_d,aroon_up,aroon_down,rsi,adx,cci,Bitcoin Price_x,Bitcoin Price_y
datetime,,,,,,,,,,,,,,,,,,,,,
2021-09-29 20:40:00+00:00,35343.300,35343.300,35316.700,35335.100,0.525724,16,-0.001171,-0.001498,-7.529013e-04,-0.000232,...,-0.837752,20.382166,28.197586,28.0,64.0,37.532442,20.561757,-104.198096,35.696296,35.696296
2018-10-24 14:08:00+00:00,5650.000,5651.000,5650.000,5650.100,0.295987,9,-0.000159,-0.000071,0.000000e+00,0.000018,...,-0.161941,5.882353,3.921569,80.0,32.0,50.000000,31.824272,-22.906441,10.804659,10.804659
2023-11-06 05:33:00+00:00,32446.400,32446.400,32435.400,32435.600,0.008196,11,0.000274,-0.000065,2.158159e-05,-0.000259,...,0.490540,40.161725,38.860960,24.0,92.0,35.483871,57.499820,-42.474213,18.871875,18.871875
2017-05-12 14:01:00+00:00,1626.001,1627.000,1626.000,1626.072,19.419248,53,-0.000627,-0.000014,-6.150060e-07,0.000044,...,0.170396,1.192053,10.809187,4.0,100.0,36.325692,38.654763,-181.837382,11.000000,11.000000
2018-06-12 13:54:00+00:00,5727.500,5728.100,5725.000,5725.000,34.101919,36,0.000035,-0.001152,-4.016660e-04,-0.000908,...,-0.130596,0.000000,18.550725,92.0,100.0,47.440273,35.979911,-99.889557,13.228056,13.228056
2020-06-27 18:57:00+00:00,8128.400,8128.400,8119.900,8119.900,0.570041,5,0.000012,0.000000,-1.033961e-03,-0.001046,...,0.228238,6.521739,68.314862,92.0,40.0,51.072961,23.573180,48.545401,9.000000,9.000000
2017-01-13 15:01:00+00:00,773.004,776.418,773.004,776.000,35.198041,29,0.000312,0.004072,3.118193e-04,0.003823,...,0.127308,94.552326,96.754534,100.0,4.0,94.171110,9.827716,164.014251,3.592720,3.592720
2023-03-22 09:42:00+00:00,26109.000,26109.000,26100.000,26100.100,0.814137,28,0.000000,0.000000,-3.409380e-04,-0.000341,...,0.258286,34.928230,56.352793,84.0,8.0,58.361775,22.554169,5.083367,21.309543,21.309543
2019-07-22 00:31:00+00:00,9475.000,9475.000,9475.000,9475.000,0.128810,1,0.000507,0.000507,9.714480e-04,0.000971,...,-4.280782,19.956616,11.281835,32.0,96.0,25.732484,44.864511,-102.717798,13.287545,13.287545


In [41]:
# Make sure your DataFrame has the necessary columns: close, high, low, volume
close = df_hourly['close']
high = df_hourly['high']
low = df_hourly['low']
volume = df_hourly['volume']

# Calculate Money Flow Multiplier (handle division by zero if high==low)
money_flow_multiplier = ((close - low) - (high - close)) / (high - low)
money_flow_multiplier = money_flow_multiplier.replace([float('inf'), -float('inf')], 0).fillna(0)

# Calculate Money Flow Volume
money_flow_volume = money_flow_multiplier * volume

# Calculate ADL by cumulative sum
df_hourly['adl'] = money_flow_volume.cumsum()

print(df_hourly['adl'].head())

datetime
2013-09-10 23:47:00+00:00    0.0
2013-09-11 00:33:00+00:00    0.0
2013-09-11 00:34:00+00:00    0.0
2013-09-11 03:22:00+00:00    0.0
2013-09-11 04:50:00+00:00   -1.0
Name: adl, dtype: float64


In [42]:
import ta

# Make sure close, high, low columns exist
close = df_hourly['close']
high = df_hourly['high']
low = df_hourly['low']

# Recompute RSI (default window=14)
df_hourly['rsi'] = ta.momentum.RSIIndicator(close=close, window=14, fillna=False).rsi()

# Recompute Stochastic %K and %D (default window=14, smooth window=3)
stoch = ta.momentum.StochasticOscillator(high=high, low=low, close=close, window=14, smooth_window=3, fillna=False)

df_hourly['stoch_k'] = stoch.stoch()
df_hourly['stoch_d'] = stoch.stoch_signal()

print(df_hourly[['rsi', 'stoch_k', 'stoch_d']].head(20))


                                 rsi  stoch_k  stoch_d
datetime                                              
2013-09-10 23:47:00+00:00        NaN      NaN      NaN
2013-09-11 00:33:00+00:00        NaN      NaN      NaN
2013-09-11 00:34:00+00:00        NaN      NaN      NaN
2013-09-11 03:22:00+00:00        NaN      NaN      NaN
2013-09-11 04:50:00+00:00        NaN      NaN      NaN
2013-09-11 07:41:00+00:00        NaN      NaN      NaN
2013-09-11 09:22:00+00:00        NaN      NaN      NaN
2013-09-13 11:16:00+00:00        NaN      NaN      NaN
2013-09-13 11:17:00+00:00        NaN      NaN      NaN
2013-09-13 11:18:00+00:00        NaN      NaN      NaN
2013-09-13 15:24:00+00:00        NaN      NaN      NaN
2013-09-13 15:27:00+00:00        NaN      NaN      NaN
2013-09-13 15:28:00+00:00        NaN      NaN      NaN
2013-09-14 16:43:00+00:00  40.943289      0.0      NaN
2013-09-14 20:52:00+00:00  33.881181      0.0      NaN
2013-09-15 05:44:00+00:00  33.881181      0.0      0.0
2013-09-15

In [43]:
# Ensure the index is a datetime index
df_hourly.index = pd.to_datetime(df_hourly.index)

# Get the first and last month (year and month)
first_month = df_hourly.index.min().to_period('M')
last_month = df_hourly.index.max().to_period('M')

# Create a mask for rows that are NOT in the first or last month AND NOT in 2015
mask = (~df_hourly.index.to_period('M').isin([first_month, last_month])) & (df_hourly.index.year != 2015)

# Apply the filter
df_filtered = df_hourly[mask]

# Result
print(df_filtered)



C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_13760\1612497797.py:5: UserWarning: Converting to Period representation will drop timezone information.
  first_month = df_hourly.index.min().to_period('M')
C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_13760\1612497797.py:6: UserWarning: Converting to Period representation will drop timezone information.
  last_month = df_hourly.index.max().to_period('M')
C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_13760\1612497797.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  mask = (~df_hourly.index.to_period('M').isin([first_month, last_month])) & (df_hourly.index.year != 2015)


                                  open         high          low        close  \
datetime                                                                        
2013-10-01 06:08:00+00:00     94.50000     94.50000     94.50000     94.50000   
2013-10-01 19:59:00+00:00     96.00000     96.00000     96.00000     96.00000   
2013-10-02 02:03:00+00:00     93.50000     93.50000     93.50000     93.50000   
2013-10-02 07:44:00+00:00     93.50000     93.50000     93.50000     93.50000   
2013-10-02 13:41:00+00:00     96.99999     96.99999     96.99999     96.99999   
...                                ...          ...          ...          ...   
2025-02-28 23:55:00+00:00  81396.10000  81396.20000  81348.80000  81348.80000   
2025-02-28 23:56:00+00:00  81344.60000  81348.60000  81334.10000  81335.00000   
2025-02-28 23:57:00+00:00  81335.00000  81335.00000  81334.10000  81334.10000   
2025-02-28 23:58:00+00:00  81334.10000  81334.10000  81334.00000  81334.00000   
2025-02-28 23:59:00+00:00  8

In [44]:

df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4833050 entries, 2013-10-01 06:08:00+00:00 to 2025-02-28 23:59:00+00:00
Data columns (total 31 columns):
 #   Column              Dtype  
---  ------              -----  
 0   open                float64
 1   high                float64
 2   low                 float64
 3   close               float64
 4   volume              float64
 5   trades              int64  
 6   open_log_returns    float64
 7   high_log_returns    float64
 8   low_log_returns     float64
 9   close_log_returns   float64
 10  volume_log_returns  float64
 11  trades_log_returns  float64
 12  hour                int64  
 13  dema                float64
 14  vwap                float64
 15  bband_up            float64
 16  bband_down          float64
 17  adl                 float64
 18  obv                 float64
 19  macd                float64
 20  macd_signal         float64
 21  macd_diff           float64
 22  stoch_k             float64
 23  stoch_d     

In [45]:
df_hourly=df_filtered

In [46]:
df_hourly['macd_signal_diffs'] = df_hourly['macd'] - df_hourly['macd_signal']


In [47]:
df_hourly['stoch'] = df_hourly['stoch_k']


In [48]:
close_ema = df_hourly['close'].ewm(span=10, adjust=False).mean()
close_ema2 = close_ema.ewm(span=10, adjust=False).mean()
df_hourly['close_dema'] = 2 * close_ema - close_ema2


In [49]:
df_hourly['close_vwap'] = (df_hourly['close'] * df_hourly['volume']).cumsum() / df_hourly['volume'].cumsum()


In [50]:
rolling_mean = df_hourly['close'].rolling(window=20).mean()
rolling_std = df_hourly['close'].rolling(window=20).std()

bband_upper = rolling_mean + 2 * rolling_std
bband_lower = rolling_mean - 2 * rolling_std

df_hourly['bband_up_close'] = bband_upper - df_hourly['close']
df_hourly['close_bband_down'] = df_hourly['close'] - bband_lower


In [51]:
df_hourly['adl_diffs2'] = df_hourly['adl'].diff(2)


In [52]:
df_hourly['obv_diffs2'] = df_hourly['obv'].diff(2)


In [53]:
# df_hourly = df_hourly.iloc[1:-1]


In [54]:
df_hourly.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 4833050 entries, 2013-10-01 06:08:00+00:00 to 2025-02-28 23:59:00+00:00
Data columns (total 39 columns):
 #   Column              Dtype  
---  ------              -----  
 0   open                float64
 1   high                float64
 2   low                 float64
 3   close               float64
 4   volume              float64
 5   trades              int64  
 6   open_log_returns    float64
 7   high_log_returns    float64
 8   low_log_returns     float64
 9   close_log_returns   float64
 10  volume_log_returns  float64
 11  trades_log_returns  float64
 12  hour                int64  
 13  dema                float64
 14  vwap                float64
 15  bband_up            float64
 16  bband_down          float64
 17  adl                 float64
 18  obv                 float64
 19  macd                float64
 20  macd_signal         float64
 21  macd_diff           float64
 22  stoch_k             float64
 23  stoch_d     

done till it 

In [95]:
df_hourly.to_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv", index=True)

In [96]:
df= pd.read_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv", index_col='datetime')

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 80324 entries, 2016-01-01 02:00:00+00:00 to 2025-02-28 21:00:00+00:00
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   open                80324 non-null  float64
 1   high                80324 non-null  float64
 2   low                 80324 non-null  float64
 3   close               80324 non-null  float64
 4   volume              80324 non-null  float64
 5   trades              80324 non-null  int64  
 6   open_log_returns    80324 non-null  float64
 7   high_log_returns    80324 non-null  float64
 8   low_log_returns     80324 non-null  float64
 9   close_log_returns   80324 non-null  float64
 10  volume_log_returns  80324 non-null  float64
 11  trades_log_returns  80324 non-null  float64
 12  hour                80324 non-null  int64  
 13  dema                80324 non-null  float64
 14  vwap                80324 non-null  float64
 15  bband_up      

In [98]:
df.head()

,open,high,low,close,volume,trades,open_log_returns,high_log_returns,low_log_returns,close_log_returns,...,cci,trends,macd_signal_diffs,stoch,close_dema,close_vwap,bband_up_close,close_bband_down,adl_diffs2,obv_diffs2
datetime,,,,,,,,,,,,,,,,,,,,,
2016-01-01 02:00:00+00:00,0.927887,0.943605,0.927887,0.943605,0.0,0,-0.000569,-0.001389,-0.000569,-0.001389,...,-120.663613,0.002688,-0.003801,48.944061,0.945041,NaN,NaN,NaN,0.0,0.0
2016-01-01 03:00:00+00:00,0.927359,0.942293,0.927359,0.942293,0.0,0,-0.000569,-0.001391,-0.000569,-0.001391,...,-117.093656,0.004032,-0.003753,47.557739,0.944056,NaN,NaN,NaN,0.0,0.0
2016-01-01 04:00:00+00:00,0.926831,0.940982,0.926831,0.940982,0.0,0,-0.000570,-0.001393,-0.000570,-0.001393,...,-116.216320,0.005376,-0.003682,46.107213,0.942905,NaN,NaN,NaN,0.0,0.0
2016-01-01 05:00:00+00:00,0.926303,0.939670,0.926303,0.939670,0.0,0,-0.000570,-0.001395,-0.000570,-0.001395,...,-117.515502,0.006720,-0.003592,44.587915,0.941637,NaN,NaN,NaN,0.0,0.0
2016-01-01 06:00:00+00:00,0.925775,0.938359,0.925775,0.938359,0.0,0,-0.000570,-0.001397,-0.000570,-0.001397,...,-111.676424,0.008065,-0.003485,42.994837,0.940290,NaN,NaN,NaN,0.0,0.0


In [99]:
# Ensure the index is a datetime index
df_hourly.index = pd.to_datetime(df_hourly.index)

# Get the first and last month (year and month)
first_month = df_hourly.index.min().to_period('M')
last_month = df_hourly.index.max().to_period('M')

# Create a mask for rows that are NOT in the first or last month AND NOT in 2015
mask = (~df_hourly.index.to_period('M').isin([first_month, last_month])) 

# Apply the filter
df_filtered = df_hourly[mask]

# Result
print(df_filtered)



                                 open        high          low        close  \
datetime                                                                      
2016-02-01 00:00:00+00:00     2.20087     2.20087     2.200870     2.200870   
2016-02-01 01:00:00+00:00     2.32125     2.32125     2.202440     2.202440   
2016-02-01 02:00:00+00:00     2.28263     2.28263     2.168293     2.168293   
2016-02-01 03:00:00+00:00     2.24401     2.24401     2.134147     2.134147   
2016-02-01 04:00:00+00:00     2.20539     2.20539     2.100000     2.100000   
...                               ...         ...          ...          ...   
2025-01-31 19:00:00+00:00  3315.00000  3343.25000  3308.070000  3341.550000   
2025-01-31 20:00:00+00:00  3338.77000  3338.77000  3286.390000  3317.750000   
2025-01-31 21:00:00+00:00  3315.24000  3319.28000  3276.820000  3315.810000   
2025-01-31 22:00:00+00:00  3315.80000  3318.29000  3282.860000  3295.620000   
2025-01-31 23:00:00+00:00  3295.34000  3300.96000  3

C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_21556\689481121.py:5: UserWarning: Converting to Period representation will drop timezone information.
  first_month = df_hourly.index.min().to_period('M')
C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_21556\689481121.py:6: UserWarning: Converting to Period representation will drop timezone information.
  last_month = df_hourly.index.max().to_period('M')
C:\Users\ujjwa_n18433z\AppData\Local\Temp\ipykernel_21556\689481121.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  mask = (~df_hourly.index.to_period('M').isin([first_month, last_month]))


In [102]:
df_filtered.to_csv(r"C:\Users\ujjwa_n18433z\Desktop\ujjwal\DAIICT\Minor Project\TraderNet-CRv2\database\storage\datasets\ETH.csv", index=True)

In [ ]:
from datetime import datetime, timedelta
import time
import random
import json

# Starting time in milliseconds since epoch (taken from your first entry)
start_time_ms = 1748708559310
interval_ms = 60000  # 1 minute interval

# Starting price values (approximation from your sample)
price = 60000

data = []

for i in range(500):
    current_time = start_time_ms + i * interval_ms
    open_price = price
    high_price = open_price + random.randint(50, 500)
    low_price = open_price - random.randint(50, 500)
    close_price = random.randint(low_price, high_price)
    price = close_price  # use close as next open

    entry = {
        "time": current_time,
        "open": open_price,
        "high": high_price,
        "low": low_price,
        "close": close_price
    }
    data.append(entry)

 # Show first 10 entries as a "preview


[{'time': 1748708559310,
  'open': 60000,
  'high': 60368,
  'low': 59684,
  'close': 60309},
 {'time': 1748708619310,
  'open': 60309,
  'high': 60745,
  'low': 59971,
  'close': 60319},
 {'time': 1748708679310,
  'open': 60319,
  'high': 60620,
  'low': 59918,
  'close': 60125},
 {'time': 1748708739310,
  'open': 60125,
  'high': 60467,
  'low': 59829,
  'close': 60194},
 {'time': 1748708799310,
  'open': 60194,
  'high': 60578,
  'low': 60124,
  'close': 60251},
 {'time': 1748708859310,
  'open': 60251,
  'high': 60672,
  'low': 60069,
  'close': 60266},
 {'time': 1748708919310,
  'open': 60266,
  'high': 60655,
  'low': 59985,
  'close': 60340},
 {'time': 1748708979310,
  'open': 60340,
  'high': 60836,
  'low': 59989,
  'close': 60647},
 {'time': 1748709039310,
  'open': 60647,
  'high': 61075,
  'low': 60234,
  'close': 60384},
 {'time': 1748709099310,
  'open': 60384,
  'high': 60733,
  'low': 60237,
  'close': 60656}]

In [1]:
import random
import json

# Starting time in milliseconds
start_time_ms = 1748708559310
interval_ms = 60000  # 1 minute interval
price = 60000

# Generate 500 entries
data = []
for i in range(500):
    current_time = start_time_ms + i * interval_ms
    open_price = price
    high_price = open_price + random.randint(50, 500)
    low_price = open_price - random.randint(50, 500)
    close_price = random.randint(low_price, high_price)
    price = close_price  # next open = current close

    data.append({
        "time": current_time,
        "open": open_price,
        "high": high_price,
        "low": low_price,
        "close": close_price
    })

# Save to a JSON file
with open("price_data_500.json", "w") as f:
    json.dump(data, f, indent=2)

print("JSON file 'price_data_500.json' created successfully.")


JSON file 'price_data_500.json' created successfully.


# New Data Set Maker 